## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Jamestown,US,42.0970,-79.2353,38.52,55,0,12.66,clear sky
1,1,Yellowknife,CA,62.4560,-114.3525,-2.58,90,100,13.80,light snow
2,2,Panzhihua,CN,26.5851,101.7128,46.33,63,0,1.12,clear sky
3,3,Karratha,AU,-20.7377,116.8463,80.29,67,96,17.20,overcast clouds
4,4,Kodiak,US,57.7900,-152.4072,26.11,74,100,4.61,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
3,3,Karratha,AU,-20.7377,116.8463,80.29,67,96,17.20,overcast clouds
7,7,Avarua,CK,-21.2078,-159.7750,78.85,83,40,17.27,scattered clouds
17,17,Atuona,PF,-9.8000,-139.0333,78.40,76,26,18.16,light rain
18,18,Rikitea,PF,-23.1203,-134.9692,78.15,76,12,13.53,light rain
28,28,Bathsheba,BB,13.2167,-59.5167,84.81,61,40,21.85,scattered clouds


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current description    0
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
3,Karratha,AU,80.29,overcast clouds,-20.7377,116.8463,
7,Avarua,CK,78.85,scattered clouds,-21.2078,-159.7750,
17,Atuona,PF,78.40,light rain,-9.8000,-139.0333,
18,Rikitea,PF,78.15,light rain,-23.1203,-134.9692,
28,Bathsheba,BB,84.81,scattered clouds,13.2167,-59.5167,
30,Castro,BR,79.36,few clouds,-24.7911,-50.0119,
36,Puerto Ayora,EC,80.56,clear sky,-0.7393,-90.3518,
39,Vaini,TO,79.23,light rain,-21.2000,-175.2000,
45,Arica,CL,83.75,clear sky,-18.4750,-70.3042,
46,Cantanhede,BR,82.38,heavy intensity rain,-3.6333,-44.3767,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}   
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():   
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [33]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=['Hotel Name'])
clean_hotel_df.head()

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
3,Karratha,AU,80.29,overcast clouds,-20.7377,116.8463,ibis Styles Karratha
7,Avarua,CK,78.85,scattered clouds,-21.2078,-159.7750,Paradise Inn
17,Atuona,PF,78.40,light rain,-9.8000,-139.0333,Villa Enata
18,Rikitea,PF,78.15,light rain,-23.1203,-134.9692,People ThankYou
28,Bathsheba,BB,84.81,scattered clouds,13.2167,-59.5167,Atlantis Hotel


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [40]:

# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
current_weather = clean_hotel_df["Current description"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))